# Random Forest Regression

## Import packages

In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.tree import plot_tree
from sklearn.metrics import (
    root_mean_squared_error,
    mean_squared_error,
    mean_absolute_error,
    r2_score,
    make_scorer,
)
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import pickle
import os

In [2]:
CLEAN_DATA_FOLDER = "clean_data"
MODELS_FOLDER = "models"

## Load the dataframe

In [3]:
train_df = pd.read_csv(os.path.join(CLEAN_DATA_FOLDER, "train_wo_weather.csv"))
test_df = pd.read_csv(os.path.join(CLEAN_DATA_FOLDER, "test_wo_weather.csv")).sort_values(
    ["Day", "Line", "Service", "Direction Number", "Sequence"]
)

## Split into X and y

In [4]:
train_X = train_df[[x for x in train_df.columns if x not in ["On", "Off"]]]
train_y = train_df["On"]
test_X = test_df[[x for x in test_df.columns if x not in ["On", "Off"]]]
test_y = test_df["On"]

## Train the Decision Tree Regressor Model

In [5]:
rf = RandomForestRegressor(
    n_estimators=70,
    max_depth=10,
    random_state=42,
    min_samples_split=14,
    min_samples_leaf=7,
    n_jobs=-1,
    verbose=1,
    criterion="poisson",
)
rf = rf.fit(X=train_X, y=train_y)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:   42.8s
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:  1.6min finished


In [6]:
train_y_pred = np.floor(rf.predict(train_X)).astype(int)
test_y_pred = np.floor(rf.predict(test_X)).astype(int)

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    0.9s
[Parallel(n_jobs=10)]: Done  70 out of  70 | elapsed:    1.9s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    0.2s
[Parallel(n_jobs=10)]: Done  70 out of  70 | elapsed:    0.4s finished


## Plot the Tree

In [7]:
# _, ax = plt.subplots(3, 1, figsize=(32, 48))
# for i in range(3):
#     _ = plot_tree(
#         rf.estimators_[i],
#         max_depth=4,
#         feature_names=train_X.columns,
#         filled=True,
#         proportion=True,
#         rounded=True,
#         precision=2,
#         fontsize=9,
#         ax=ax[i],
#     )

## Feature Importance

In [8]:
# feat_imp = pd.DataFrame(
#     {
#         "Feature": [x for x in rf.feature_names_in_],
#         "Importance": [x for x in rf.feature_importances_],
#     }
# )
# _, ax = plt.subplots(1, 1, figsize=(16, 9))
# _ = sns.barplot(feat_imp, x="Feature", y="Importance")
# _ = plt.title("Feature Importance for Decision Tree Regressor")

## Visualize the Predictions

In [9]:
# line_fit = pd.DataFrame({"True": test_y, "Predicted": test_y_pred}, index=test_df["Day"])
# _, ax = plt.subplots(1, 1, figsize=(16,9))
# _ = sns.lineplot(line_fit, legend=True, ax=ax)

## Report Train and Test results

In [10]:
print("train rmse:", root_mean_squared_error(train_y, train_y_pred))
print("train mae:", mean_absolute_error(train_y, train_y_pred))
print("train r2 score:", r2_score(train_y, train_y_pred))

train rmse: 30.934791360158556
train mae: 10.94878166260619
train r2 score: 0.6115099744184215


In [11]:
print("test rmse:", root_mean_squared_error(test_y, test_y_pred))
print("test mae:", mean_absolute_error(test_y, test_y_pred))
print("test r2 score:", r2_score(test_y, test_y_pred))

test rmse: 28.081743561123297
test mae: 10.524957402204127
test r2 score: 0.6619704324788698


## Export Model

In [12]:
pickle.dump(rf, open(os.path.join(MODELS_FOLDER, "base_random_forest_wo_weather.pkl"), "wb"))

In [13]:
del rf

## Hyperparameter Tuning with GridSearchCV

| n_estimators | max_depth | max_features | criterion |
| --- | --- | --- | --- |
| 100 | 10, 20 | None | squared_error, poisson |
| 100 | 70, None | sqrt | squared_error, poisson |
| 10 | 70 | 1.0 | squared_error, poisson |
| 10 | None | sqrt | squared_error, poisson |

### Declare base model and parameters

In [14]:
base_rf = RandomForestRegressor(random_state=42, min_samples_split=14, min_samples_leaf=7)
param_grid = [
    {
        "criterion": ["squared_error", "poisson"],
        "n_estimators": [50],
        "max_depth": [20],
        "max_features": [1.0],
    },
    {
        "criterion": ["squared_error", "poisson"],
        "n_estimators": [50],
        "max_depth": [100],
        "max_features": ["sqrt"],
    },
    {
        "criterion": ["squared_error", "poisson"],
        "n_estimators": [10],
        "max_depth": [70],
        "max_features": [1.0],
    },
    {
        "criterion": ["squared_error", "poisson"],
        "n_estimators": [10],
        "max_depth": [100],
        "max_features": ["sqrt"],
    },
]

### Declare the scorer and grid search

In [15]:
scorer = make_scorer(mean_squared_error, greater_is_better=False)
grid_search = GridSearchCV(base_rf, param_grid, scoring=scorer, n_jobs=-1, verbose=2, cv=3)

### Train the models

In [16]:
grid_search.fit(train_X, train_y)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] END criterion=squared_error, max_depth=100, max_features=sqrt, n_estimators=50; total time= 5.7min
[CV] END criterion=squared_error, max_depth=100, max_features=sqrt, n_estimators=50; total time= 5.7min
[CV] END criterion=squared_error, max_depth=100, max_features=sqrt, n_estimators=50; total time= 5.8min
[CV] END criterion=poisson, max_depth=100, max_features=sqrt, n_estimators=50; total time= 6.4min
[CV] END criterion=squared_error, max_depth=70, max_features=1.0, n_estimators=10; total time= 2.4min
[CV] END criterion=squared_error, max_depth=70, max_features=1.0, n_estimators=10; total time= 2.5min
[CV] END criterion=squared_error, max_depth=20, max_features=1.0, n_estimators=50; total time=10.3min
[CV] END criterion=squared_error, max_depth=20, max_features=1.0, n_estimators=50; total time=10.5min
[CV] END criterion=squared_error, max_depth=20, max_features=1.0, n_estimators=50; total time=10.6min
[CV] END criterion=s

GridSearchCV(cv=3,
             estimator=RandomForestRegressor(min_samples_leaf=7,
                                             min_samples_split=14,
                                             random_state=42),
             n_jobs=-1,
             param_grid=[{'criterion': ['squared_error', 'poisson'],
                          'max_depth': [20], 'max_features': [1.0],
                          'n_estimators': [50]},
                         {'criterion': ['squared_error', 'poisson'],
                          'max_depth': [100], 'max_features': ['sqrt'],
                          'n_estimators': [50]},
                         {'criterion': ['squared_error', 'poisson'],
                          'max_depth': [70], 'max_features': [1.0],
                          'n_estimators': [10]},
                         {'criterion': ['squared_error', 'poisson'],
                          'max_depth': [100], 'max_features': ['sqrt'],
                          'n_estimators': [10]}],
             scoring=make_scorer(mean_squared_error, greater_is_better=False, response_method='predict'),
             verbose=2)

In [17]:
pd.DataFrame(grid_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,602.794499,8.265170,25.800391,0.940742,squared_error,20,1.0,50,"{'criterion': 'squared_error', 'max_depth': 20...",-685.790624,-662.915438,-647.905371,-665.537144,15.577293,5
1,651.928058,0.441929,24.669217,0.608777,poisson,20,1.0,50,"{'criterion': 'poisson', 'max_depth': 20, 'max...",-687.533795,-664.573856,-648.550200,-666.885950,15.998740,6
2,277.666399,1.255515,67.074951,0.591440,squared_error,100,sqrt,50,"{'criterion': 'squared_error', 'max_depth': 10...",-673.016402,-649.128365,-637.239386,-653.128051,14.877205,2
3,313.662988,2.310650,67.841541,1.504260,poisson,100,sqrt,50,"{'criterion': 'poisson', 'max_depth': 100, 'ma...",-673.242843,-646.757672,-637.084825,-652.361780,15.284089,1
4,131.673235,4.726622,16.365380,0.702658,squared_error,70,1.0,10,"{'criterion': 'squared_error', 'max_depth': 70...",-689.839628,-674.373900,-659.633752,-674.615760,12.332683,8
5,133.738137,11.185666,9.084421,3.636278,poisson,70,1.0,10,"{'criterion': 'poisson', 'max_depth': 70, 'max...",-687.947720,-674.780106,-657.516847,-673.414891,12.460801,7
6,49.798219,2.809465,12.336314,1.614427,squared_error,100,sqrt,10,"{'criterion': 'squared_error', 'max_depth': 10...",-683.233847,-662.272000,-649.598347,-665.034731,13.869901,4
7,51.960730,2.275841,8.900959,0.828615,poisson,100,sqrt,10,"{'criterion': 'poisson', 'max_depth': 100, 'ma...",-684.064962,-657.944723,-646.909689,-662.973125,15.579734,3


In [18]:
print(grid_search.best_params_)

{'criterion': 'poisson', 'max_depth': 100, 'max_features': 'sqrt', 'n_estimators': 50}


### Extract the best model

In [19]:
best_rf = grid_search.best_estimator_

In [20]:
train_y_pred = np.floor(best_rf.predict(train_X)).astype(int)
test_y_pred = np.floor(best_rf.predict(test_X)).astype(int)

### Plot the Tree

In [21]:
# _, ax = plt.subplots(3, 1, figsize=(32, 48))
# for i in range(3):
#     _ = plot_tree(
#         best_rf.estimators_[i],
#         max_depth=4,
#         feature_names=train_X.columns,
#         filled=True,
#         proportion=True,
#         rounded=True,
#         precision=2,
#         fontsize=9,
#         ax=ax[i],
#     )

### Feature Importance

In [22]:
# feat_imp = pd.DataFrame(
#     {
#         "Feature": [x for x in best_rf.feature_names_in_],
#         "Importance": [x for x in best_rf.feature_importances_],
#     }
# )
# _, ax = plt.subplots(1, 1, figsize=(16, 9))
# _ = sns.barplot(feat_imp, x="Feature", y="Importance")
# _ = plt.title("Feature Importance for Best Decision Tree Regressor")

### Visualize the Predictions

In [23]:
# line_fit = pd.DataFrame({"True": test_y, "Predicted": test_y_pred}, index=test_df["Day"])
# _, ax = plt.subplots(1, 1, figsize=(16,9))
# _ = sns.lineplot(line_fit, legend=True, ax=ax)

### Report Train and Test results

In [24]:
print("train rmse:", root_mean_squared_error(train_y, train_y_pred))
print("train mae:", mean_absolute_error(train_y, train_y_pred))
print("train r2 score:", r2_score(train_y, train_y_pred))

train rmse: 22.720250036566366
train mae: 5.953004221413614
train r2 score: 0.7904384011342858


In [25]:
print("test rmse:", root_mean_squared_error(test_y, test_y_pred))
print("test mae:", mean_absolute_error(test_y, test_y_pred))
print("test r2 score:", r2_score(test_y, test_y_pred))

test rmse: 26.330031201955194
test mae: 7.258849920391005
test r2 score: 0.7028270338521301


### Export Model

In [26]:
pickle.dump(best_rf, open(os.path.join(MODELS_FOLDER, "tuned_random_forest_wo_weather.pkl"), "wb"))